## 1. Import libraries

In [169]:
import requests
from selenium import webdriver
from time import sleep
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import pandas as pd
import sys
sys.path.append('../')
import numpy as np
import re
import folium

## 2. Import Data

In [2]:
web_df = pd.read_csv("../Data/Web_scraping.csv")

## 3. We explore the dataset

In [3]:
web_df.head(3)

,Unnamed: 0,Climate,Growth,GDP_per_capita,Unemployment,Homosexuality_acceptance,LGBT_adoption_rights,LGBT_homosexuality_rights,LGBT_marriage_rights,VAT_Sales_Tax,...,Gun_related_deaths,Guns_per_residents,Pisa_ranking,Best_university,Large_apartment,Medium_apartment,Small_apartment,Description,Poblacion,Cities
0,0,No hay dato,5%,"$44,343",4%,93.03,✔ Equal,✔ Sexual orientation and gender identity,0.99,0.25,...,1,12,13,Aarhus University,$1700,$1300,$840,"Aarhus, Denmark, is among the top cities with ...","250,200",Aarhus
1,1,12.6,7%,"$46,433",6%,0.79,NaN,✔ Legal,Varies by Region,0.10,...,1,24,16,University of Adelaide,$1500,$1200,$1000,"Adelaide, Australia, is among the top cities w...","1,708,700",Adelaide
2,2,168,7%,"$54,597",5%,93.13,✔ Equal,✔ Sexual orientation and gender identity,0.45,0.07,...,16,328,24,University of New Mexico,$1200,$950,$760,"Albuquerque, New Mexico, is among the top citi...","630,800",Albuquerque


In [4]:
web_df.shape

(262, 33)

In [5]:
web_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 262 entries, 0 to 261
Data columns (total 33 columns):
 #   Column                           Non-Null Count  Dtype 
---  ------                           --------------  ----- 
 0   Unnamed: 0                       262 non-null    int64 
 1   Climate                          262 non-null    object
 2   Growth                           262 non-null    object
 3   GDP_per_capita                   262 non-null    object
 4   Unemployment                     262 non-null    object
 5   Homosexuality_acceptance         262 non-null    object
 6   LGBT_adoption_rights             254 non-null    object
 7   LGBT_homosexuality_rights        262 non-null    object
 8   LGBT_marriage_rights             262 non-null    object
 9   VAT_Sales_Tax                    262 non-null    object
 10  Art_galleries                    262 non-null    object
 11  Concert_venues                   262 non-null    object
 12  Museums                          262

#### We can see that most of the data are object type, so we will have to convert the columns to their respective dtypes.

In [6]:
pd.isna(web_df).sum()


Unnamed: 0                         0
Climate                            0
Growth                             0
GDP_per_capita                     0
Unemployment                       0
Homosexuality_acceptance           0
LGBT_adoption_rights               8
LGBT_homosexuality_rights          0
LGBT_marriage_rights               0
VAT_Sales_Tax                      0
Art_galleries                      0
Concert_venues                     0
Museums                            0
Sport_venues                       0
Currency_for_urban_area            0
GDP_per_capita.1                   0
Monthly_Fitness_Club_Membership    0
Beer                               0
Monthly_Public_Transport           0
Lunch                              0
Airport_score                      0
Airport_hub                        1
Intercity_train_connectivity       1
Gun_related_deaths                 1
Guns_per_residents                 1
Pisa_ranking                       1
Best_university                    1
L

## 4. Data cleaning

#### Some of the values present are nan, so we will have to replace the information in those places.

In [7]:
web_df.LGBT_adoption_rights.unique()

array(['✔ Equal', nan, 'Ambiguous', '✖ Unequal', '✖ Single only',
       'Married couples only', 'No hay dato', '✔ Legal', '✖ Illegal',
       '✖ Step-child adoption only'], dtype=object)

In [8]:
web_df.Airport_hub.unique() 

array(['8', '26', '23', '45', '237', '38', '0.10', '42', '10', '104',
       '217', '44', '39', '0.48', '139', '163', '215', '51', '40', '49',
       '142', '152', '102', '76', '16', '64', '103', '169', '7', '149',
       '176', '59', '80', '1', '189', '66', '77', '67', '24', '81', '135',
       '22', '46', '87', '20', '141', '25', '48', '56', '18', '11', '34',
       '126', '30', '190', '97', '17', '137', '116', '144', '117', '79',
       '9', '78', '12', '2', '239', '47', '101', '41', '19', '28', '90',
       '50', '31', '88', '54', '171', '32', '35', '6', '242', '27', '43',
       '52', '115', '71', '133', '159', '155', '160', '357', '62', '158',
       '85', '94', '167', '164', '130', '93', '257', '83', '191', '72',
       '55', nan, '220', '91', '61', '146', '123', '3', '294',
       'No hay dato', '82', '317', '15', '121', '57', '36', '68', '109',
       '111', '4', '29', '96', '13', '95', '180', '134', '33', '125',
       '98', '113', '143', '148', '60', '86', '21'], dtype=objec

In [9]:
web_df.Intercity_train_connectivity.unique()

array(['0.59', '0.17', '0.11', '0.66', '0.68', 'No hay dato', '0.16',
       '0.12', '0.26', '0.15', '0.20', '0.14', '0.21', '0.46', '0.50',
       '0.57', '0.37', '0.78', '0.70', '0.69', '0.40', '0.33', '0.42',
       '0.36', '0.41', '0.35', '0.19', '0.60', '0.13', '0.73', '0.27',
       '0.67', '0.55', '0.32', '0.61', '0.48', '0.71', '1.00', '0.30',
       '0.10', '0.85', '0.18', '0.53', '0.56', '0.24', '0.72', '0.47',
       '0.83', '0.63', '0.64', '0.88', '0.81', '0.39', '0.51', '0.25',
       '0.80', nan, '0.86', '0.44', '0.65', '0.22', '0.62', '0.29',
       '0.28', '0.76', '0.43', '0.95', '0.94'], dtype=object)

In [10]:
web_df.Gun_related_deaths.unique()

array(['1', '16', '20', '2', '12', '8', '13', '11', '0', '10', '3', '26',
       '14', '17', '6', '4', '33', '59', '9', '21', '34', '15', '31', '5',
       '25', nan, 'No hay dato', '46'], dtype=object)

In [11]:
web_df.Guns_per_residents.unique()

array(['12', '24', '328', '1', '4', '135', '17', '13', '95', '23', '118',
       '123', '119', '16', '10', '5', '21', '6', '58', '31', '30', '185',
       '35', '103', '112', '8', '26', '11', '108', '96', '80', '7', '91',
       '52', '39', '19', '22', '93', '15', '27', '2', '115', '98', '76',
       '3', '168', '9', '14', '61', '120', '69', '85', '32', nan, '36',
       '121', '70', 'No hay dato', '129', '154', '29', '246', '130', '0',
       '491'], dtype=object)

In [12]:
web_df.Pisa_ranking.unique()

array(['13', '16', '24', '39', '10', 'No hay dato', '43', '35', '12',
       '48', '47', '22', '8', '51', '17', '0.39', '11', '26', '19', '34',
       '21', '15', '40', '31', '5', '9', '52', '49', '41', '3', '14', '6',
       '2', '20', '38', '18', '25', '46', nan, '28', '37', '7', '1', '36',
       '4', '32', '30'], dtype=object)

In [13]:
#web_df.Best_university.unique()

In [14]:
web_df['LGBT_adoption_rights'] = web_df['LGBT_adoption_rights'].replace(np.nan, "No hay dato")

In [15]:
web_df['Airport_hub'] = web_df['Airport_hub'].replace(np.nan, 0)

In [16]:
web_df["Gun_related_deaths"] = web_df["Gun_related_deaths"].replace(np.nan, 0)

In [17]:
web_df["Guns_per_residents"] = web_df["Guns_per_residents"].replace(np.nan, 0)

In [18]:
web_df["Pisa_ranking"] = web_df["Pisa_ranking"].replace(np.nan, 0)

In [19]:
web_df["Best_university"] = web_df["Best_university"].replace(np.nan,'No hay dato')

In [20]:
web_df["Intercity_train_connectivity"] = web_df["Intercity_train_connectivity"].replace(np.nan, 0)

In [21]:
pd.isna(web_df).sum()


Unnamed: 0                         0
Climate                            0
Growth                             0
GDP_per_capita                     0
Unemployment                       0
Homosexuality_acceptance           0
LGBT_adoption_rights               0
LGBT_homosexuality_rights          0
LGBT_marriage_rights               0
VAT_Sales_Tax                      0
Art_galleries                      0
Concert_venues                     0
Museums                            0
Sport_venues                       0
Currency_for_urban_area            0
GDP_per_capita.1                   0
Monthly_Fitness_Club_Membership    0
Beer                               0
Monthly_Public_Transport           0
Lunch                              0
Airport_score                      0
Airport_hub                        0
Intercity_train_connectivity       0
Gun_related_deaths                 0
Guns_per_residents                 0
Pisa_ranking                       0
Best_university                    0
L

### We will generate a copy to manipulate and work with it. 

In [22]:
copia = web_df.copy()

In [23]:
copia.head()

,Unnamed: 0,Climate,Growth,GDP_per_capita,Unemployment,Homosexuality_acceptance,LGBT_adoption_rights,LGBT_homosexuality_rights,LGBT_marriage_rights,VAT_Sales_Tax,...,Gun_related_deaths,Guns_per_residents,Pisa_ranking,Best_university,Large_apartment,Medium_apartment,Small_apartment,Description,Poblacion,Cities
0,0,No hay dato,5%,"$44,343",4%,93.03,✔ Equal,✔ Sexual orientation and gender identity,0.99,0.25,...,1,12,13,Aarhus University,$1700,$1300,$840,"Aarhus, Denmark, is among the top cities with ...","250,200",Aarhus
1,1,12.6,7%,"$46,433",6%,0.79,No hay dato,✔ Legal,Varies by Region,0.10,...,1,24,16,University of Adelaide,$1500,$1200,$1000,"Adelaide, Australia, is among the top cities w...","1,708,700",Adelaide
2,2,168,7%,"$54,597",5%,93.13,✔ Equal,✔ Sexual orientation and gender identity,0.45,0.07,...,16,328,24,University of New Mexico,$1200,$950,$760,"Albuquerque, New Mexico, is among the top citi...","630,800",Albuquerque
3,3,64,9%,"$24,020",6%,66.56,✔ Equal,Ambiguous,0.64,0.12,...,1,1,39,427.97,$620,$480,$340,"Almaty, Kazakhstan, differentiates itself with...","2,029,800",Almaty
4,4,12.9,3%,"$47,355",6%,97.37,✔ Equal,✔ Sexual orientation and gender identity,0.67,0.21,...,1,4,10,University of Amsterdam,$2400,$2000,$1600,Amsterdam is one of the most bike-friendly cit...,"2,628,100",Amsterdam


In [24]:
copia.columns

Index(['Unnamed: 0', 'Climate', 'Growth', 'GDP_per_capita', 'Unemployment',
       'Homosexuality_acceptance', 'LGBT_adoption_rights',
       'LGBT_homosexuality_rights', 'LGBT_marriage_rights', 'VAT_Sales_Tax',
       'Art_galleries', 'Concert_venues', 'Museums', 'Sport_venues',
       'Currency_for_urban_area', 'GDP_per_capita.1',
       'Monthly_Fitness_Club_Membership', 'Beer', 'Monthly_Public_Transport',
       'Lunch', 'Airport_score', 'Airport_hub', 'Intercity_train_connectivity',
       'Gun_related_deaths', 'Guns_per_residents', 'Pisa_ranking',
       'Best_university', 'Large_apartment', 'Medium_apartment',
       'Small_apartment', 'Description', 'Poblacion', 'Cities'],
      dtype='object')

### To be able to work with the data, we must remove symbols and clean the columns

In [25]:
def delete_price(i):
    i = str(i).replace('$', "")
    return i

In [26]:
copia['GDP_per_capita'] = copia['GDP_per_capita'].apply(delete_price)

In [27]:
copia['Monthly_Fitness_Club_Membership'] = copia['Monthly_Fitness_Club_Membership'].apply(delete_price)

In [28]:
copia['Beer'] = copia['Beer'].apply(delete_price)

In [29]:
copia['Monthly_Public_Transport'] = copia['Monthly_Public_Transport'].apply(delete_price)

In [30]:
copia['Lunch'] = copia['Lunch'].apply(delete_price)

In [31]:
copia['Large_apartment'] = copia['Large_apartment'].apply(delete_price)

In [32]:
copia['Medium_apartment'] = copia['Medium_apartment'].apply(delete_price)

In [33]:
copia['Small_apartment'] = copia['Small_apartment'].apply(delete_price)

In [34]:
def convertir(i):
    i = i.replace(",", ".")
    return i

In [35]:
copia["GDP_per_capita"] = copia["GDP_per_capita"].apply(convertir)

In [36]:
def quitar(i):
    i = i.replace(",", "")
    return i

In [37]:
copia["Poblacion"] = copia["Poblacion"].apply(quitar)

In [38]:
def clean_symbol(i):
    i = i.replace("✖", "")
    return i

In [39]:
copia["LGBT_adoption_rights"] = copia["LGBT_adoption_rights"].apply(clean_symbol)

In [40]:
copia["LGBT_homosexuality_rights"] = copia["LGBT_homosexuality_rights"].apply(clean_symbol)

In [41]:
copia["LGBT_marriage_rights"] = copia["LGBT_marriage_rights"].apply(clean_symbol)

In [42]:
def clean_symbol2(i):
    i = i.replace("✔", "")
    return i

In [43]:
copia["LGBT_adoption_rights"] = copia["LGBT_adoption_rights"].apply(clean_symbol2)

In [44]:
copia["LGBT_homosexuality_rights"] = copia["LGBT_homosexuality_rights"].apply(clean_symbol2)

In [45]:
copia["LGBT_marriage_rights"] = copia["LGBT_marriage_rights"].apply(clean_symbol2)

In [72]:
copia.head(3)

,Unnamed: 0,Climate,Growth,GDP_per_capita,Unemployment,Homosexuality_acceptance,LGBT_adoption_rights,LGBT_homosexuality_rights,LGBT_marriage_rights,VAT_Sales_Tax,...,Gun_related_deaths,Guns_per_residents,Pisa_ranking,Best_university,Large_apartment,Medium_apartment,Small_apartment,Description,Poblacion,Cities
0,0,0,5%,44.343,4%,93.03,Equal,Sexual orientation and gender identity,0.99,0.25,...,1,12,13,Aarhus University,1700,1300,840,"Aarhus, Denmark, is among the top cities with ...",250200,Aarhus
1,1,12.6,7%,46.433,6%,0.79,No hay dato,Legal,Varies by Region,0.10,...,1,24,16,University of Adelaide,1500,1200,1000,"Adelaide, Australia, is among the top cities w...",1708700,Adelaide
2,2,168,7%,54.597,5%,93.13,Equal,Sexual orientation and gender identity,0.45,0.07,...,16,328,24,University of New Mexico,1200,950,760,"Albuquerque, New Mexico, is among the top citi...",630800,Albuquerque


In [47]:
def clean_no_info(i):
    i = i.replace("No hay dato", "0")
    return i

In [48]:
copia["Airport_hub"] = copia["Airport_hub"].replace("No hay dato", 0)

In [49]:
copia["Intercity_train_connectivity"] = copia["Intercity_train_connectivity"].replace("No hay dato", 0)

In [50]:
copia["Gun_related_deaths"] = copia["Gun_related_deaths"].replace("No hay dato", 0)

In [51]:
copia["Guns_per_residents"] = copia["Guns_per_residents"].replace("No hay dato", 0)

In [52]:
copia["Pisa_ranking"] = copia["Pisa_ranking"].replace("No hay dato", 0)

In [53]:
copia["Large_apartment"] = copia["Large_apartment"].replace("No hay dato", 0)

In [54]:
copia["Small_apartment"] = copia["Small_apartment"].replace("No hay dato", 0)

In [55]:
copia["Medium_apartment"] = copia["Medium_apartment"].replace("No hay dato", 0)

In [56]:
copia["VAT_Sales_Tax"] = copia["VAT_Sales_Tax"].replace("No hay dato", 0)

In [57]:
copia["Art_galleries"] = copia["Art_galleries"].replace("No hay dato", 0)

In [58]:
copia["Concert_venues"] = copia["Concert_venues"].replace("No hay dato", 0)

In [59]:
copia["Museums"] = copia["Museums"].replace("No hay dato", 0)

In [60]:
copia["Sport_venues"] = copia["Sport_venues"].replace("No hay dato", 0)

In [61]:
copia["GDP_per_capita"] = copia["GDP_per_capita"].replace("No hay dato", 0)

In [62]:
copia["Unemployment"] = copia["Unemployment"].replace("No hay dato", 0)

In [63]:
copia["Climate"] = copia["Climate"].replace("No hay dato", 0)

In [64]:
copia["Homosexuality_acceptance"] = copia["Homosexuality_acceptance"].replace("No hay dato", 0)

In [65]:
copia["Airport_score"] = copia["Airport_score"].replace("No hay dato", 0)

In [101]:
copia["Monthly_Fitness_Club_Membership"] = copia["Monthly_Fitness_Club_Membership"].replace("No hay dato", 0)

In [105]:
copia["Lunch"] = copia["Lunch"].replace("No hay dato", 0)

In [106]:
copia["Beer"] = copia["Beer"].replace("No hay dato", 0)

In [110]:
copia["Monthly_Public_Transport"] = copia["Monthly_Public_Transport"].replace("No hay dato", 0)

In [67]:
def replace_num(universidad):
    #print ("i original :",universidad)
    if (re.findall("\d+",universidad)):
        universidad = "No hay dato"
    #print ("i cambiada: ",universidad)
    return universidad

In [68]:
copia["Best_university"] = copia["Best_university"].apply(replace_num)

In [69]:
def replace_num2(Currency):
    if (re.findall("\d+",Currency)):
        Currency = "USD"
    return Currency

In [70]:
copia.loc[copia['Currency_for_urban_area'] =='0.43']

,Unnamed: 0,Climate,Growth,GDP_per_capita,Unemployment,Homosexuality_acceptance,LGBT_adoption_rights,LGBT_homosexuality_rights,LGBT_marriage_rights,VAT_Sales_Tax,...,Gun_related_deaths,Guns_per_residents,Pisa_ranking,Best_university,Large_apartment,Medium_apartment,Small_apartment,Description,Poblacion,Cities
16,16,0,80,0,0,96.88,No hay dato,No hay dato,No hay dato,0,...,10,119,24,Johns Hopkins University,1900,1600,1200,This urban area is consistently ahead in healt...,38,Baltimore


In [71]:
copia["Currency_for_urban_area"] = copia["Currency_for_urban_area"].apply(replace_num2)

#### Some of the columns have data that does not correspond to the measurement in that column, so we will replace those numbers with 0, giving it the assignment that we have no data for that location.

In [73]:
def change(x):
    x = str(x).split('.')[0]
    return x

In [74]:
copia["Airport_hub"] = copia["Airport_hub"].apply(change) 

In [75]:
copia["Pisa_ranking"] =copia["Pisa_ranking"].apply(change) 

In [76]:
copia["Concert_venues"] = copia["Concert_venues"].apply(change)

In [77]:
copia["Museums"] = copia["Museums"].apply(change)

In [78]:
copia["Sport_venues"] = copia["Sport_venues"].apply(change)

In [79]:
def delete_percentage(i):
    i = str(i).replace('%', "")
    return i

In [80]:
copia["Growth"] = copia["Growth"].apply(delete_percentage)

In [81]:
copia["Unemployment"] = copia["Unemployment"].apply(delete_percentage)

In [118]:
copia["GDP_per_capita"] = copia["GDP_per_capita"].apply(delete_percentage)

### We must convert the dtype of the columns

In [82]:
copia.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 262 entries, 0 to 261
Data columns (total 33 columns):
 #   Column                           Non-Null Count  Dtype 
---  ------                           --------------  ----- 
 0   Unnamed: 0                       262 non-null    int64 
 1   Climate                          262 non-null    object
 2   Growth                           262 non-null    object
 3   GDP_per_capita                   262 non-null    object
 4   Unemployment                     262 non-null    object
 5   Homosexuality_acceptance         262 non-null    object
 6   LGBT_adoption_rights             262 non-null    object
 7   LGBT_homosexuality_rights        262 non-null    object
 8   LGBT_marriage_rights             262 non-null    object
 9   VAT_Sales_Tax                    262 non-null    object
 10  Art_galleries                    262 non-null    object
 11  Concert_venues                   262 non-null    object
 12  Museums                          262

In [83]:
copia["Small_apartment"] = copia["Small_apartment"].astype(dtype=int)

In [84]:
copia["Medium_apartment"] = copia["Medium_apartment"].astype(dtype=int)

In [85]:
copia["Large_apartment"] = copia["Large_apartment"].astype(dtype=int)

In [86]:
copia["Airport_hub"] = copia["Airport_hub"].astype(dtype=int)

In [87]:
copia["Gun_related_deaths"] = copia["Gun_related_deaths"].astype(dtype=int)

In [88]:
copia["Guns_per_residents"] = copia["Guns_per_residents"].astype(dtype=int)

In [89]:
copia["Pisa_ranking"] = copia["Pisa_ranking"].astype(dtype=int)

In [90]:
copia["Poblacion"] = copia["Poblacion"].astype(dtype=int)

In [91]:
copia["Art_galleries"] = copia["Art_galleries"].astype(dtype=int)

In [92]:
copia["Concert_venues"] = copia["Concert_venues"].astype(dtype=int)

In [93]:
copia["Museums"] = copia["Museums"].astype(dtype=int)

In [94]:
copia["Sport_venues"] = copia["Sport_venues"].astype(dtype=int)

In [95]:
copia["Growth"] = copia["Growth"].astype(dtype=int)

In [96]:
copia["Unemployment"] = copia["Unemployment"].astype(dtype=int)

In [99]:
copia["Intercity_train_connectivity"] = copia["Intercity_train_connectivity"].astype(dtype=float)

In [102]:
copia["Monthly_Fitness_Club_Membership"] = copia["Monthly_Fitness_Club_Membership"].astype(dtype=float)

In [107]:
copia["Beer"] = copia["Beer"].astype(dtype=float)

In [108]:
copia["Lunch"] = copia["Lunch"].astype(dtype=float)

In [111]:
copia["Monthly_Public_Transport"] = copia["Monthly_Public_Transport"].astype(dtype=float)

In [112]:
copia["Homosexuality_acceptance"] = copia["Homosexuality_acceptance"].astype(dtype=float)

In [113]:
copia["VAT_Sales_Tax"] = copia["VAT_Sales_Tax"].astype(dtype=float)

In [115]:
copia["Climate"] = copia["Climate"].astype(dtype=float)

In [119]:
copia["GDP_per_capita"] =copia["GDP_per_capita"].astype(dtype=float)

In [120]:
copia["Airport_score"] = copia["Airport_score"].astype(dtype=float)

In [121]:
def porcentaje(i):
    i = i/100
    return i

In [122]:
copia["Growth"] = copia["Growth"].apply(porcentaje)

In [123]:
copia["Unemployment"] = copia["Unemployment"].apply(porcentaje)

In [125]:
def menores(i):
    if i <1:
        i= i*100
    return i

In [126]:
copia["Homosexuality_acceptance"] = copia["Homosexuality_acceptance"].astype(dtype=float)

In [127]:
copia["Homosexuality_acceptance"] = copia["Homosexuality_acceptance"].apply(menores)

In [128]:
copia.head(3)

,Unnamed: 0,Climate,Growth,GDP_per_capita,Unemployment,Homosexuality_acceptance,LGBT_adoption_rights,LGBT_homosexuality_rights,LGBT_marriage_rights,VAT_Sales_Tax,...,Gun_related_deaths,Guns_per_residents,Pisa_ranking,Best_university,Large_apartment,Medium_apartment,Small_apartment,Description,Poblacion,Cities
0,0,0.0,0.05,44.343,0.04,93.03,Equal,Sexual orientation and gender identity,0.99,0.25,...,1,12,13,Aarhus University,1700,1300,840,"Aarhus, Denmark, is among the top cities with ...",250200,Aarhus
1,1,12.6,0.07,46.433,0.06,79.00,No hay dato,Legal,Varies by Region,0.10,...,1,24,16,University of Adelaide,1500,1200,1000,"Adelaide, Australia, is among the top cities w...",1708700,Adelaide
2,2,168.0,0.07,54.597,0.05,93.13,Equal,Sexual orientation and gender identity,0.45,0.07,...,16,328,24,University of New Mexico,1200,950,760,"Albuquerque, New Mexico, is among the top citi...",630800,Albuquerque


In [130]:
copia.columns

Index(['Unnamed: 0', 'Climate', 'Growth', 'GDP_per_capita', 'Unemployment',
       'Homosexuality_acceptance', 'LGBT_adoption_rights',
       'LGBT_homosexuality_rights', 'LGBT_marriage_rights', 'VAT_Sales_Tax',
       'Art_galleries', 'Concert_venues', 'Museums', 'Sport_venues',
       'Currency_for_urban_area', 'GDP_per_capita.1',
       'Monthly_Fitness_Club_Membership', 'Beer', 'Monthly_Public_Transport',
       'Lunch', 'Airport_score', 'Airport_hub', 'Intercity_train_connectivity',
       'Gun_related_deaths', 'Guns_per_residents', 'Pisa_ranking',
       'Best_university', 'Large_apartment', 'Medium_apartment',
       'Small_apartment', 'Description', 'Poblacion', 'Cities'],
      dtype='object')

### A continuación eliminaremos algunas columnas que no utilizaremos

In [131]:
copia.drop(["Unnamed: 0", "GDP_per_capita.1"], axis=1, inplace=True)

In [133]:
new_name = {"Growth": "Growth_percentage", "Unemployment": "Unemployment_percentage"}

In [134]:
copia.rename(columns=new_name, inplace=True)

In [135]:
copia.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 262 entries, 0 to 261
Data columns (total 31 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   Climate                          262 non-null    float64
 1   Growth_percentage                262 non-null    float64
 2   GDP_per_capita                   262 non-null    float64
 3   Unemployment_percentage          262 non-null    float64
 4   Homosexuality_acceptance         262 non-null    float64
 5   LGBT_adoption_rights             262 non-null    object 
 6   LGBT_homosexuality_rights        262 non-null    object 
 7   LGBT_marriage_rights             262 non-null    object 
 8   VAT_Sales_Tax                    262 non-null    float64
 9   Art_galleries                    262 non-null    int64  
 10  Concert_venues                   262 non-null    int64  
 11  Museums                          262 non-null    int64  
 12  Sport_venues          

In [136]:
copia.to_csv("../Data/Data_clean_user.csv")

In [ ]:
copia.columns

### Agregaremos coordenadas para hacer mapas de cada ciudad

In [137]:
import pandas as pd
import xlrd
from geopy.geocoders import Nominatim

In [138]:
def get_coordenadas(city):
    """
    Esta función saca las coordenadas de la ciudad que le pases.
    Args: una ciudad (string).
    Return: Las coordeandas de la ciudad que le paso como argumento (latitud y longitud).
    """
    geolocator = Nominatim(user_agent="Dani")
    location = geolocator.geocode(query=city, exactly_one=True)
    
    return location[1]

In [139]:
copia['Coordenadas'] = copia['Cities'].apply(get_coordenadas)

In [143]:
def hacer_lista(resp):
    return list(resp)

In [144]:
copia['Coordenadas'] = copia['Coordenadas'].apply(hacer_lista)

In [146]:
type(coord_feas)

list

In [147]:
type(copia.Coordenadas[0][0])

float

In [162]:
def latitud(data):
    return data[0]

In [165]:
def longitud(data):
    return data[1]

In [163]:
copia["lat"] = copia.Coordenadas.apply(latitud)

In [166]:
copia["long"] = copia.Coordenadas.apply(longitud)

In [167]:
copia.head()

,Climate,Growth_percentage,GDP_per_capita,Unemployment_percentage,Homosexuality_acceptance,LGBT_adoption_rights,LGBT_homosexuality_rights,LGBT_marriage_rights,VAT_Sales_Tax,Art_galleries,...,Best_university,Large_apartment,Medium_apartment,Small_apartment,Description,Poblacion,Cities,Coordenadas,lat,long
0,0.0,0.05,44.343,0.04,93.03,Equal,Sexual orientation and gender identity,0.99,0.25,51,...,Aarhus University,1700,1300,840,"Aarhus, Denmark, is among the top cities with ...",250200,Aarhus,"[56.1496278, 10.2134046]",56.149628,10.213405
1,12.6,0.07,46.433,0.06,79.00,No hay dato,Legal,Varies by Region,0.10,82,...,University of Adelaide,1500,1200,1000,"Adelaide, Australia, is among the top cities w...",1708700,Adelaide,"[-34.9281805, 138.5999312]",-34.928181,138.599931
2,168.0,0.07,54.597,0.05,93.13,Equal,Sexual orientation and gender identity,0.45,0.07,127,...,University of New Mexico,1200,950,760,"Albuquerque, New Mexico, is among the top citi...",630800,Albuquerque,"[35.0841034, -106.650985]",35.084103,-106.650985
3,64.0,0.09,24.020,0.06,66.56,Equal,Ambiguous,0.64,0.12,61,...,No hay dato,620,480,340,"Almaty, Kazakhstan, differentiates itself with...",2029800,Almaty,"[43.2363924, 76.9457275]",43.236392,76.945728
4,12.9,0.03,47.355,0.06,97.37,Equal,Sexual orientation and gender identity,0.67,0.21,284,...,University of Amsterdam,2400,2000,1600,Amsterdam is one of the most bike-friendly cit...,2628100,Amsterdam,"[52.3727598, 4.8936041]",52.372760,4.893604


In [168]:
copia.to_csv("../Data/user_coord.csv")

### Con nuestros datos limpios, le daremos forma a un nuevo dataset que utilizaremos para realizar los clusters 

In [ ]:
PCA_user = copia.copy()

In [ ]:
PCA_user.columns

#### Eliminamos columnas que no utilizaremos

In [ ]:
PCA_user.drop(["Airport_hub","Guns_per_residents", "Best_university", "Monthly_Fitness_Club_Membership","Beer","Monthly_Public_Transport","Lunch","Description","Poblacion","LGBT_adoption_rights","LGBT_homosexuality_rights","LGBT_marriage_rights","Url","Currency_for_urban_area"], axis=1, inplace=True)

In [ ]:
PCA_user.head()

In [ ]:
PCA_user['Housing'] = PCA_user[["Large_apartment", "Medium_apartment","Small_apartment"]].mean(axis=1)

In [ ]:
PCA_user.drop(["Large_apartment","Medium_apartment","Small_apartment"], axis=1, inplace=True)

In [ ]:
PCA_user['Culture'] = PCA_user[['Art_galleries','Concert_venues','Museums']].sum(axis=1)

In [ ]:
PCA_user.drop(["Art_galleries","Concert_venues","Museums"], axis=1, inplace=True)

In [ ]:
PCA_user.info()

In [ ]:
PCA_user['Travel_connectivity'] = PCA_user[["Intercity_train_connectivity", "Airport_score"]].mean(axis=1)

In [ ]:
PCA_user.drop(["Intercity_train_connectivity","Airport_score"], axis=1, inplace=True)

In [ ]:
PCA_user["Housing"] = PCA_user["Housing"].astype(dtype=int)

In [ ]:
PCA_user.head()

In [ ]:
PCA_user.to_csv("../Data/clean_PCA_user.csv")